In [1]:
from time import time

import numpy as np
import pandas as pd

from lift.backend.comparison import (
    ComparisonChargerType,
    ComparisonChargingInfrastructure,
    ComparisonESS,
    ComparisonFix,
    ComparisonFixedDemand,
    ComparisonFleet,
    ComparisonGrid,
    ComparisonPV,
    ComparisonResult,
    ComparisonScenario,
    ComparisonSettings,
    ComparisonSubFleet,
)
from lift.backend.scenario import ExistExpansionValue, SingleScenario
from lift.frontend.definitions import DEF_CIS, DEF_ESS, DEF_FLEET, DEF_GRID, DEF_PV, DEF_SCN

In [2]:
def value_factor_product(x):
    return x.value * x.factor


start = time()

comp_scn = ComparisonScenario(
    settings=ComparisonSettings(
        latitude=48.1351,
        longitude=11.5820,
        wacc=value_factor_product(DEF_SCN.wacc),
        period_eco=int(DEF_SCN.period_eco.value),
        sim_start=DEF_SCN.sim_start,
        sim_duration=DEF_SCN.sim_duration,
        sim_freq=DEF_SCN.sim_freq,
    ),
    fix=ComparisonFix(
        capex_initial=ExistExpansionValue(
            baseline=0.0,
            expansion=value_factor_product(DEF_SCN.capex_initial),
        ),
        capem_initial=ExistExpansionValue(
            baseline=DEF_SCN.capem_initial,
            expansion=DEF_SCN.capem_initial,
        ),
    ),
    dem=ComparisonFixedDemand(
        slp=DEF_SCN.slp.value.lower(),
        e_yrl=value_factor_product(DEF_SCN.e_yrl),
    ),
    grid=ComparisonGrid(
        capacity=ExistExpansionValue(
            baseline=value_factor_product(DEF_GRID.capacity_preexisting),
            expansion=value_factor_product(DEF_GRID.capacity_preexisting)
            + value_factor_product(DEF_GRID.capacity_expansion),
        ),
        opex_spec_buy=value_factor_product(DEF_GRID.opex_spec_buy),
        opex_spec_sell=value_factor_product(DEF_GRID.opex_spec_sell),
        opex_spec_peak=value_factor_product(DEF_GRID.opex_spec_peak),
        **DEF_GRID.values,
    ),
    pv=ComparisonPV(
        capacity=ExistExpansionValue(
            baseline=value_factor_product(DEF_PV.capacity_preexisting),
            expansion=value_factor_product(DEF_PV.capacity_preexisting)
            + value_factor_product(DEF_PV.capacity_expansion),
        ),
        **DEF_PV.values,
    ),
    ess=ComparisonESS(
        capacity=ExistExpansionValue(
            baseline=value_factor_product(DEF_ESS.capacity_preexisting),
            expansion=value_factor_product(DEF_ESS.capacity_preexisting)
            + value_factor_product(DEF_ESS.capacity_expansion),
        ),
        **DEF_ESS.values,
    ),
    fleet=ComparisonFleet(
        subblocks={
            k: ComparisonSubFleet(
                name=k,
                num_bev=ExistExpansionValue(
                    baseline=0,
                    expansion=0,
                ),
                num_icev=ExistExpansionValue(
                    baseline=0,
                    expansion=0,
                ),
                charger="ac",
                p_max=11 * 1e3,
                capex_per_unit_bev=value_factor_product(DEF_FLEET.subblocks[k].capex_per_unit_bev),
                capex_per_unit_icev=value_factor_product(DEF_FLEET.subblocks[k].capex_per_unit_icev),
                toll_frac=value_factor_product(DEF_FLEET.subblocks[k].toll_frac),
                **DEF_FLEET.subblocks[k].values,
            )
            for k in DEF_FLEET.subblocks.keys()
        },
        opex_spec_fuel=value_factor_product(DEF_FLEET.opex_spec_fuel),
        opex_spec_onroute_charging=value_factor_product(DEF_FLEET.opex_spec_onroute_charging),
        **DEF_FLEET.values,
    ),
    cis=ComparisonChargingInfrastructure(
        p_lm_max=ExistExpansionValue(
            baseline=11e3,
            expansion=11e3,
        ),
        subblocks={
            k: ComparisonChargerType(
                name=k,
                num=ExistExpansionValue(baseline=0, expansion=0),
                p_max=11 * 1e3,
                capex_per_unit=800,
                **DEF_CIS.subblocks[k].values,
            )
            for k in DEF_CIS.subblocks.keys()
        },
    ),
)

scn_dict = {
    "baseline": SingleScenario.from_comparison(comp_scn, phase="baseline"),
    "expansion": SingleScenario.from_comparison(comp_scn, phase="expansion"),
}

results = {scn_name: scn_obj.simulate() for scn_name, scn_obj in scn_dict.items()}

comp_result = ComparisonResult(baseline=results["baseline"], expansion=results["expansion"])

df_cost_comparison = pd.DataFrame({k: np.cumsum(v.totex_dis) for k, v in results.items()})

print(f"Simulation completed in {time() - start:.2f} seconds.")

Scenario simulation time: 0.0331 s
Scenario simulation time: 0.0319 s
Simulation completed in 18.39 seconds.
